In [ ]:
import json
import torch
from transformers import Qwen3Config
from nandmachine.frontend.network.qwen3 import Qwen3DecoderLayer
from nandmachine.frontend.core.graph.base import NxTracer
from nandmachine.frontend.network.torch_kernels import *